TED talk text generator

Generate using a character based model.

The code is this notebook is a partial modification of several tutorials, blog posts and YouTube videos. The main sources include:
* [Text Generation With LSTM Recurrent Neural Networks in Python with Keras](https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/) by Dr. Jason Brownlee
* [Text Generation with Python and TensorFlow/Keras](https://stackabuse.com/text-generation-with-python-and-tensorflow-keras/) by Dan Nelson
* YouTube: [167 - Text prediction using LSTM (English text)](https://youtu.be/zyCpntcVKSo?t=2) by Sreenivas B.


In [115]:
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.train import latest_checkpoint
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Dropout, LSTM
import random
import textwrap

In [116]:
# Attach to my Google drive so I can save the csv file later
# from google.colab import drive
# drive.mount('/content/drive')

In [117]:
#tag_name = 'all_transcripts'
#tag_name = 'brain'
tag_name = 'business'
#tag_name = 'creativity'
#tag_name = 'culture'
#tag_name = 'psychology'
#tag_name = 'science'

lower_corpus = False

if lower_corpus:
    corpus_file_name = tag_name + '_lowercase'
else:
    corpus_file_name = tag_name + '_sentence_case'

# This switch makes it easier running locally or in a Google Colab environment
colab = False

if colab:
    root_path = '/content/drive/MyDrive/ted/'
else:
    root_path = '../data/'
    
sequence_length = 80

In [118]:
with open(f'{root_path}{corpus_file_name}.txt', encoding='utf-8') as f:
    corpus = f.read()
f.close()

In [119]:
corpus[0:500]

"So I want to start by offering you a free no-tech life hack, and all it requires of you is  that you change your posture for two minutes. But before I give it away, I want to ask you to right now do a little audit of your body and what you're doing with your body. So how many of you are sort of making yourselves smaller? Maybe you're hunching, crossing your legs, maybe wrapping your ankles. Sometimes we hold onto our arms like this. Sometimes we spread out.  I see you. So I want you to pay atten"

The code below is used in almost every article I have seen about LSTM for text generation, but this snippet is adapted from :
https://www.kaggle.com/mrisdal/intro-to-lstms-w-keras-gpu-for-text-generation/

In [120]:
all_chars = sorted(list(set(corpus)))

#remove most punctuation, but leave spaces, exclamation points, and periods
list_to_remove = ['"', '&', '-', ',', "'", '/', ';',  '—']

chars = [character for character in all_chars if character not in list_to_remove]
# print(chars)
# # hat tip to this post for the idea of removing one list from another
# # https://www.geeksforgeeks.org/python-remove-all-values-from-a-list-present-in-other-list/

In [121]:
# remove all unwanted characters from the original corpus
corpus_list = [character for character in corpus if character not in list_to_remove]

# Re-join all letters back into a single string as the updated corpus
corpus = ''.join(corpus_list)

In [122]:
# If running models with punctuation removed, leave commented out
# chars = sorted(list(set(corpus)))

print('Number of unique characters in the corpus:', len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))

indices_char = dict((i, c) for i, c in enumerate(chars))

Number of unique characters in the corpus: 66


In [123]:
# Show character to index mapping
print (char_indices)

{' ': 0, '!': 1, '%': 2, '.': 3, '0': 4, '1': 5, '2': 6, '3': 7, '4': 8, '5': 9, '6': 10, '7': 11, '8': 12, '9': 13, '?': 14, 'A': 15, 'B': 16, 'C': 17, 'D': 18, 'E': 19, 'F': 20, 'G': 21, 'H': 22, 'I': 23, 'J': 24, 'K': 25, 'L': 26, 'M': 27, 'N': 28, 'O': 29, 'P': 30, 'Q': 31, 'R': 32, 'S': 33, 'T': 34, 'U': 35, 'V': 36, 'W': 37, 'Y': 38, 'Z': 39, 'a': 40, 'b': 41, 'c': 42, 'd': 43, 'e': 44, 'f': 45, 'g': 46, 'h': 47, 'i': 48, 'j': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'q': 56, 'r': 57, 's': 58, 't': 59, 'u': 60, 'v': 61, 'w': 62, 'x': 63, 'y': 64, 'z': 65}


In [124]:
# Show index to character mapping
print(indices_char)

{0: ' ', 1: '!', 2: '%', 3: '.', 4: '0', 5: '1', 6: '2', 7: '3', 8: '4', 9: '5', 10: '6', 11: '7', 12: '8', 13: '9', 14: '?', 15: 'A', 16: 'B', 17: 'C', 18: 'D', 19: 'E', 20: 'F', 21: 'G', 22: 'H', 23: 'I', 24: 'J', 25: 'K', 26: 'L', 27: 'M', 28: 'N', 29: 'O', 30: 'P', 31: 'Q', 32: 'R', 33: 'S', 34: 'T', 35: 'U', 36: 'V', 37: 'W', 38: 'Y', 39: 'Z', 40: 'a', 41: 'b', 42: 'c', 43: 'd', 44: 'e', 45: 'f', 46: 'g', 47: 'h', 48: 'i', 49: 'j', 50: 'k', 51: 'l', 52: 'm', 53: 'n', 54: 'o', 55: 'p', 56: 'q', 57: 'r', 58: 's', 59: 't', 60: 'u', 61: 'v', 62: 'w', 63: 'x', 64: 'y', 65: 'z'}


In [125]:
print (f'The input corpus contains {len(corpus):,} characters.')

The input corpus contains 68,253 characters.


The model will use arbitrary length of characters, e.g. 40, and then predict the next character that will appear (the 41st).

I would like to change this sequence length in different models, to see how it might affect the model's performance.

Larger might be better?

Loop through the corpus, creating segments of 40 characters, plus a segment of a single character that would appear after it. These segments are then converted into their respective digits, and loaded into X

In addition, create a list of target values. The target, y, is a single text character, and is also converted into a corresponding numeric value.

In [126]:
corpus[0:50]

'So I want to start by offering you a free notech l'

In [127]:
corpus[1:41]

'o I want to start by offering you a free'

In [128]:
corpus[41]

' '

In [129]:
# For future modeling, we could try to offset the sequences by more than one
# letter. For now the model will look at every sequence right after another.
step = 1

X_numeric_list = []
y_numeric_list = []


for i in range (0, len(corpus) - sequence_length, step):
    # To find X when still as characters, loop through and extract
    # a sequence for example, from the 2nd to the 42nd, as in corpus[1:41] since 
    # the string index is zero-based
    X_char_sequence = corpus[i:i + sequence_length]  #exclusive
    y_char = corpus[i + sequence_length]

    # Convert the X character sequence into a list of integers, using the 
    # dictionary created above.
    X_numeric_list.append( [char_indices[letter] for letter in X_char_sequence])
    # Also convert target letter y to it's corresponding numeric value in the
    # dictionary
    y_numeric_list.append(char_indices[y_char])



In [130]:
# Look at the first two converted sequences
print ( X_numeric_list[0])
print ( X_numeric_list[1])


[33, 54, 0, 23, 0, 62, 40, 53, 59, 0, 59, 54, 0, 58, 59, 40, 57, 59, 0, 41, 64, 0, 54, 45, 45, 44, 57, 48, 53, 46, 0, 64, 54, 60, 0, 40, 0, 45, 57, 44, 44, 0, 53, 54, 59, 44, 42, 47, 0, 51, 48, 45, 44, 0, 47, 40, 42, 50, 0, 40, 53, 43, 0, 40, 51, 51, 0, 48, 59, 0, 57, 44, 56, 60, 48, 57, 44, 58, 0, 54]
[54, 0, 23, 0, 62, 40, 53, 59, 0, 59, 54, 0, 58, 59, 40, 57, 59, 0, 41, 64, 0, 54, 45, 45, 44, 57, 48, 53, 46, 0, 64, 54, 60, 0, 40, 0, 45, 57, 44, 44, 0, 53, 54, 59, 44, 42, 47, 0, 51, 48, 45, 44, 0, 47, 40, 42, 50, 0, 40, 53, 43, 0, 40, 51, 51, 0, 48, 59, 0, 57, 44, 56, 60, 48, 57, 44, 58, 0, 54, 45]


In [131]:
print ( y_numeric_list[0:2])

[45, 0]


In [132]:
indices_char[51]

'l'

In [133]:
corpus[0:43]

'So I want to start by offering you a free n'

This example is a little hard to follow, since we happened to be in the middle of the word 'been'. The letter e is mapped to 51.

However the beginning of the sequence has been shifted by one as well.

In [134]:
print (f'There are {len(X_numeric_list):,} entries in X_numeric_list') 

There are 68,173 entries in X_numeric_list


An LSTM model needs the X data to be in 3 dimensions:
* Samples (number of rows)
* Time steps (this implies time series. In this example this corresponds to the sequence_length)
* Features (the target, or y variable)

This [blog post by Dr. John Brownlee](https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/) contains a longer explanation, which I paraphrased here.

In [135]:
# reshape X to be [samples, time steps, features]
# Samples is the total number of 40 character text fragments created above
# Time step is the sequence length, which is this case is 40, but might change
# in other model variations
# Feature is one, since the model is predicting one character at a time
X = np.reshape(X_numeric_list, (len(X_numeric_list), sequence_length, 1))

In [136]:
X.shape

(68173, 80, 1)

In [137]:
# normalize/scale the data by dividing by the length of the character list.
X = X / len(chars)

In [153]:
# Look at the first value of X
#X[0]

Even thought the values for the target variable are integers, they are essentially labels for the predicted characters. They are one-hot encoded using the to_categorical function.

In [139]:
y = to_categorical(y_numeric_list)
y[0]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
      dtype=float32)

---
### Load LSTM Model
In this notebook, load a model that was previously trained and saved.

In [140]:
# Save the model to my Google Drive so I can load it later from another notebook
# model = load_model(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}')

# model.summary()

If a model failed during fitting, we can still load it's most recent weights which were stored in a checkpoint file.

Use of ModelCheckPoint to save the model's progress example as described in the [TensorFlow documentation.](https://www.tensorflow.org/tutorials/keras/save_and_load#checkpoint_callback_options)

In [141]:
checkpoint_dir = f'{root_path}{corpus_file_name}/'

latest = latest_checkpoint(checkpoint_dir)


In [142]:
# On Colab, model saved as 
# model.save(f'/content/drive/MyDrive/ted/models/ted_model_{tag_name}_seq_{sequence_length}')

In [143]:
print ( f'{root_path}models/ted_model_{tag_name}')

../data/models/ted_model_business


In [144]:
# define the LSTM model
# model = Sequential()
# model.add(LSTM(1000, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
# model.add(Dropout(0.2))
# model.add(LSTM(256))
# model.add(Dropout(0.2))
# model.add(Dense(y.shape[1], activation='softmax'))


# # Load the previously saved weights
# model.load_weights(latest)

# model.compile(loss='categorical_crossentropy', optimizer='adam' , metrics = ['accuracy','Recall','Precision'])
#model = load_model(f'{latest}')

# Use if there was a successful save of a model


#model = load_model(f'{root_path}models/ted_model_{tag_name}')
model = load_model(f'{root_path}/models/ted_model_{tag_name}')

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 80, 100)           40800     
_________________________________________________________________
dropout_2 (Dropout)          (None, 80, 100)           0         
_________________________________________________________________
lstm_3 (LSTM)                (None, 128)               117248    
_________________________________________________________________
dropout_3 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 66)                8514      
Total params: 166,562
Trainable params: 166,562
Non-trainable params: 0
_________________________________________________________________


X_numeric_list contains all of our 40 character sequences. Choose one for a seed for the text generation.

In [145]:
seed_start = random.randint(0, len(corpus) - sequence_length)
print(f'Random start index for seed text: {seed_start} sequence.')

X_seeded = X_numeric_list[seed_start]
print(X_seeded)

Random start index for seed text: 9114 sequence.
[61, 44, 51, 0, 40, 53, 43, 0, 40, 51, 58, 54, 0, 59, 47, 40, 59, 0, 57, 54, 51, 44, 0, 42, 47, 40, 53, 46, 44, 58, 0, 42, 40, 53, 0, 58, 47, 40, 55, 44, 0, 59, 47, 44, 0, 52, 48, 53, 43, 3, 0, 33, 54, 0, 62, 47, 40, 59, 0, 47, 40, 55, 55, 44, 53, 58, 0, 54, 50, 40, 64, 0, 64, 54, 60, 0, 59, 40, 50, 44]


Convert this back to text so we can see what it looks like.

In [146]:
seed_string = ''.join([indices_char[letter_code] for letter_code in X_seeded])
seed_string

'vel and also that role changes can shape the mind. So what happens okay you take'

To generate text, we need to change the input text into a 3D shape as we did the X input data.

In [147]:
temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
# normalize/scale the data by dividing by the length of the character list.
temp_x = temp_x / len(chars)
prediction = model.predict(temp_x, verbose=0)
print(f"A prediction's shape:{prediction.shape}")
print('Prediction:')
print (prediction)

A prediction's shape:(1, 66)
Prediction:
[[9.6023411e-01 3.6121673e-06 1.1453022e-15 1.5051023e-02 1.1256549e-15
  2.2267809e-11 7.8010709e-11 6.0025139e-14 2.2914915e-16 1.9633632e-14
  4.5014583e-14 2.5579229e-24 5.5918828e-15 1.9282237e-16 2.0589097e-03
  3.4238010e-12 1.2886770e-13 2.0849728e-14 2.1221766e-11 1.6299131e-10
  2.0263212e-10 7.1127099e-13 1.7985463e-13 1.9795242e-07 5.1315935e-16
  3.1515199e-19 1.0535313e-11 8.4331681e-11 5.2911614e-14 4.0347048e-10
  1.0707417e-13 2.4099792e-23 7.0585631e-15 2.6025366e-11 4.0938449e-13
  3.2888788e-15 1.2451486e-10 1.2516598e-08 9.4741767e-12 9.0302020e-24
  7.2955161e-05 1.2253222e-04 4.3681208e-05 1.7929057e-03 3.8078558e-04
  7.9270467e-06 2.9920586e-05 1.3755137e-04 4.1783541e-03 3.1948221e-07
  5.4178305e-05 3.2516857e-04 9.1172209e-05 3.8952224e-03 1.4868609e-04
  1.7311604e-05 2.5032219e-07 6.8856077e-04 9.6915243e-03 1.2774025e-04
  1.2002273e-04 3.6602105e-05 7.1815055e-05 1.7357801e-08 6.1677763e-04
  1.3470685e-07]]


Each prediction produces an result of shape (1, 75), and contains a predicted probability of a character. Use argmax to find the index of the highest probability.

In [148]:
np.argmax(prediction)

0

So the first letter predicted after the seed sequence is:

In [149]:
indices_char[np.argmax(prediction)]

' '

In [150]:
# Put all predicted characters in a list, then convert to a string later.
complete_predicted_text =[]

# Generate 1000 characters
for i in range(600):
    # reshape to 1 row, sequence length, and 1 for predicting one character as the target
    temp_x = np.reshape(X_seeded, (1, len(X_seeded) , 1))
    # normalize/scale the data by dividing by the length of the character list.
    temp_x = temp_x / len(chars)
    prediction = model.predict(temp_x, verbose=0)
    complete_predicted_text.append(indices_char[np.argmax(prediction)])

    # After predicting, add this character's numeric value on to the randomly
    # chose seed string, and use this for another prediction
    X_seeded.append(np.argmax(prediction))

    # Move the seeded text up one character, so create a new sequence
    X_seeded = X_seeded[1:len(X_seeded)]


#print (complete_predicted_text)


The predicted results are a list of characters, so convert into a string

In [151]:
predicted_string = ''.join(complete_predicted_text)

# Wrap to 120 characters just so I can visually compare to the original string.
textwrap.wrap(seed_string + '->'+ predicted_string, width=120)

['vel and also that role changes can shape the mind. So what happens okay you take-> the powerful  So the second the same',
 'things to the pert of the boimg the bright bround the mand of the canele thiy wene people  Its a compurer comfinent',
 'They have the sert of the boise the tame things they wene people  They have the sereered  The seaond the same things',
 'they wene people  They have the sereered  The seaond the same things they wene people  They have the sereered  The',
 'seaond the same things they wene people  They have the sereered  The seaond the same things they wene people  They have',
 'the sereered  The seaond the same things they wene people  They have the sereered  The seaond']

In [152]:
# Find the original seed string to see what came after it
original_paragraph = corpus[corpus.find(seed_string): corpus.find(seed_string) + 2000]

textwrap.wrap(original_paragraph, width=120)

['vel and also that role changes can shape the mind. So what happens okay you take a role change what happens if you do',
 'that at a really minimal level like this tiny manipulation this tiny intervention? For two minutes you say I want you to',
 'stand like this and its going to make you feel more powerful. So this is what we did. We decided to bring people into',
 'the lab and run a little experiment and these people adopted for two minutes either highpower poses or lowpower poses',
 'and Im just going to show you five of the poses although they took on only two. So heres one. A couple more. This one',
 'has been dubbed the Wonder Woman by the media. Here are a couple more. So you can be standing or you can be sitting. And',
 'here are the lowpower poses. So youre folding up youre making yourself small. This one is very lowpower. When youre',
 'touching your neck youre really protecting yourself. So this is what happens. They come in they spit into a vial for two',
 'minutes we say 